In [196]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # elimina los errres de que van a cambiar

# importamos las librerías que necesitamos
# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
import re

In [197]:
df = pd.read_csv("hr_raw_data_limpio_.csv", index_col=0)

In [198]:
df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,RoleDepartament,RemoteWork
0,51,No,NaN,684.0,NaN,6,High School,NaN,162.0,Very Low,Male,51.0,Medium,Director,Research Director,Medium,NaN,19537.0,6462,7,No,13,"3,0",Medium,NaN,None,NaN,5,"3,0",20,15,15,NaN,Yes
1,52,No,NaN,699.0,NaN,1,Bachelor's,Life Sciences,259.0,Medium,Male,65.0,Low,Director,Manager,Medium,NaN,19999.0,5678,0,NaN,14,"3,0",Very Low,NaN,Low,34.0,5,"3,0",33,11,9,NaN,Yes
2,42,No,Travel Rarely,532.0,Research & Development,4,Secondary,Technical Degree,319.0,Medium,Male,58.0,Medium,Director,Manager,High,Married,19232.0,4933,1,No,11,"3,0",High,NaN,None,22.0,3,NaN,22,11,15,Manager Research & Development,Yes
3,47,No,Travel Rarely,359.0,NaN,2,Bachelor's,Medical,NaN,Very Low,Female,82.0,Medium,Manager,Research Director,Medium,Married,17169.0,26703,3,NaN,19,"3,0",Low,NaN,Medium,NaN,2,NaN,20,5,6,NaN,No
4,46,No,NaN,1319.0,NaN,3,High School,Technical Degree,NaN,Very Low,Female,45.0,High,Manager,Sales Executive,Very Low,Divorced,NaN,7739,2,No,12,"3,0",High,NaN,Low,NaN,5,"3,0",19,2,8,NaN,No


In [206]:
df[["MonthlyRate", "MonthlyIncome","DailyRate","OverTime","StandardHours","EmployeeNumber"]].dropna(subset=['EmployeeNumber']).sort_values(by='EmployeeNumber').head(10)

,MonthlyRate,MonthlyIncome,DailyRate,OverTime,StandardHours,EmployeeNumber
263,24907,NaN,NaN,No,80.0,2.0
1226,2396,2090.0,1373.0,Yes,NaN,4.0
264,23159,2909.0,1392.0,NaN,NaN,5.0
797,16632,NaN,591.0,No,80.0,7.0
1227,9964,NaN,1324.0,NaN,NaN,10.0
522,12682,NaN,153.0,Yes,NaN,15.0
798,15170,NaN,670.0,NaN,NaN,16.0
799,8758,NaN,1346.0,NaN,NaN,18.0
800,12947,NaN,103.0,Yes,NaN,19.0
107,10195,NaN,1389.0,No,NaN,20.0


In [212]:
df1= df[df.duplicated('EmployeeNumber', keep=False)].dropna(subset=['EmployeeNumber']).sort_values(by='EmployeeNumber')

In [278]:
media_80[["MonthlyRate", "MonthlyIncome","DailyRate","HourlyRate","OverTime","StandardHours","EmployeeNumber","JobRole"]].iloc[20:30]

,MonthlyRate,MonthlyIncome,DailyRate,HourlyRate,OverTime,StandardHours,EmployeeNumber,JobRole
78,15830,NaN,288.0,99.0,No,80.0,117.0,Healthcare Representative
80,5615,NaN,1354.0,45.0,NaN,80.0,153.0,Manager
86,10942,9613.0,1132.0,48.0,No,80.0,NaN,Healthcare Representative
88,4345,NaN,982.0,59.0,No,80.0,NaN,Research Director
89,4156,4306.0,480.0,31.0,No,80.0,524.0,Manufacturing Director
95,17970,NaN,1050.0,65.0,NaN,80.0,1117.0,Sales Executive
109,13982,6347.0,1240.0,48.0,Yes,80.0,184.0,Human Resources
112,11012,3072.0,111.0,66.0,No,80.0,239.0,Laboratory Technician
121,9278,NaN,549.0,75.0,No,80.0,452.0,Research Scientist
128,6950,NaN,319.0,56.0,NaN,80.0,NaN,Manufacturing Director


In [257]:
df["MonthlyRate"].isna().sum()

0

In [279]:
media_80 = df[df['StandardHours'] == 80]
media_nan = df[df['StandardHours'].isna()]
print(media_80["MonthlyRate"].median())
print(media_nan["MonthlyRate"].median())
print(media_80["MonthlyIncome"].median())
print(media_nan["MonthlyIncome"].median())
print(media_80["DailyRate"].median())
print(media_nan["DailyRate"].median())
print(media_80["HourlyRate"].median())
print(media_nan["HourlyRate"].median())

14180.0
14295.0
5103.0
4768.0
788.5
800.5
67.5
66.0


In [253]:
media_80["OverTime"].value_counts()

No     169
Yes     69
Name: OverTime, dtype: int64

In [280]:
nombre_df = media_80.groupby(by="JobRole")["MonthlyRate"].agg(["mean", "median", "min", "max"]).reset_index()
nombre_df

,JobRole,mean,median,min,max
0,Healthcare Representative,13716.324324,14075.0,2112,25949
1,Human Resources,12780.900000,12203.5,2097,23965
2,Laboratory Technician,13096.256757,11894.5,3372,26062
3,Manager,16801.531250,19089.5,3156,26582
4,Manufacturing Director,13887.710526,14071.0,2689,26767
5,Research Director,15404.291667,16803.0,2288,25258
6,Research Scientist,13732.614458,13072.0,2125,26427
7,Sales Executive,15594.544444,14954.0,2253,26841
8,Sales Representative,14419.476190,14947.0,2539,23522


In [281]:
nombre_df2 = media_nan.groupby(by="JobRole")["MonthlyRate"].agg(["mean", "median", "min", "max"]).reset_index()
nombre_df2

,JobRole,mean,median,min,max
0,Healthcare Representative,14285.883929,14395.0,2094,26542
1,Human Resources,13837.729730,11585.0,2243,26894
2,Laboratory Technician,14487.333333,14445.5,2104,26914
3,Manager,14600.721519,15896.0,2437,26362
4,Manufacturing Director,14800.091667,14669.0,2323,26493
5,Research Director,14963.234375,15576.5,2125,26862
6,Research Scientist,13969.554113,13588.0,2122,26999
7,Sales Executive,13954.974910,13430.0,2137,26997
8,Sales Representative,14011.159420,12388.0,2755,26956


In [ ]:
df[df.duplicated('EmployeeNumber', keep=False)].dropna(subset=['EmployeeNumber']).sort_values(by='EmployeeNumber').iloc[50:55]

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,RoleDepartament,RemoteWork
882,21,Yes,NaN,756.0,NaN,31,Primary,NaN,478.0,Very Low,Female,99.0,Low,Entry Level,Sales Representative,Low,Single,2174.0,9150,1,NaN,11,"3,0",Medium,NaN,None,3.0,3,"3,0",3,1,2,NaN,Yes
1523,21,Yes,NaN,756.0,NaN,1,Primary,NaN,478.0,Very Low,Female,99.0,Low,Entry Level,Sales Representative,Low,Single,2174.0,9150,1,NaN,11,"3,0",Medium,NaN,None,3.0,3,"3,0",3,1,2,NaN,No
883,57,No,Travel Rarely,593.0,NaN,1,Bachelor'S,Medical,482.0,High,Male,88.0,Medium,Assistant,Healthcare Representative,Medium,NaN,6755.0,2967,2,No,11,"3,0",Medium,NaN,None,NaN,2,"3,0",3,1,2,NaN,Yes
1524,57,No,Travel Rarely,593.0,NaN,1,Bachelor'S,Medical,482.0,High,Male,88.0,Medium,Assistant,Healthcare Representative,Medium,NaN,6755.0,2967,2,No,11,"3,0",Medium,NaN,None,NaN,2,"3,0",3,1,2,NaN,No
1558,40,No,Travel Rarely,1171.0,NaN,10,Bachelor'S,Life Sciences,483.0,High,Female,46.0,High,Entry Level,Laboratory Technician,Medium,Married,NaN,22495,3,Yes,13,"3,0",Medium,NaN,Low,NaN,3,"3,0",7,1,7,NaN,Yes


In [191]:
df[df.duplicated(keep=False)].sort_values(by='EmployeeNumber').shape

(102, 34)

In [192]:
df[df.duplicated('EmployeeNumber', keep=False)].dropna(subset=['EmployeeNumber']).sort_values(by='EmployeeNumber').shape

(208, 34)

In [193]:
df[df.duplicated('EmployeeNumber', keep=False)].dropna(subset=['EmployeeNumber']).sort_values(by='EmployeeNumber').iloc[50:55]

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome,MonthlyRate,NumCompaniesWorked,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsSinceLastPromotion,YearsWithCurrManager,RoleDepartament,RemoteWork
882,21,Yes,NaN,756.0,NaN,31,Primary,NaN,478.0,Very Low,Female,99.0,Low,Entry Level,Sales Representative,Low,Single,2174.0,9150,1,NaN,11,"3,0",Medium,NaN,None,3.0,3,"3,0",3,1,2,NaN,Yes
1523,21,Yes,NaN,756.0,NaN,1,Primary,NaN,478.0,Very Low,Female,99.0,Low,Entry Level,Sales Representative,Low,Single,2174.0,9150,1,NaN,11,"3,0",Medium,NaN,None,3.0,3,"3,0",3,1,2,NaN,No
883,57,No,Travel Rarely,593.0,NaN,1,Bachelor'S,Medical,482.0,High,Male,88.0,Medium,Assistant,Healthcare Representative,Medium,NaN,6755.0,2967,2,No,11,"3,0",Medium,NaN,None,NaN,2,"3,0",3,1,2,NaN,Yes
1524,57,No,Travel Rarely,593.0,NaN,1,Bachelor'S,Medical,482.0,High,Male,88.0,Medium,Assistant,Healthcare Representative,Medium,NaN,6755.0,2967,2,No,11,"3,0",Medium,NaN,None,NaN,2,"3,0",3,1,2,NaN,No
1558,40,No,Travel Rarely,1171.0,NaN,10,Bachelor'S,Life Sciences,483.0,High,Female,46.0,High,Entry Level,Laboratory Technician,Medium,Married,NaN,22495,3,Yes,13,"3,0",Medium,NaN,Low,NaN,3,"3,0",7,1,7,NaN,Yes


In [199]:
df["MaritalStatus"].value_counts()

Married     439
Single      325
Divorced    199
Name: MaritalStatus, dtype: int64